# CORD-19-analyse-collected-data

In general, this jupyter notebook is designated to analyse additional colllected scopus data. Thusly, an API-call is described: 
https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Thusly, the fetched coredata and affiliation are read from the pkl-files and combined to a DataFrame for further processing. 

In [2]:
read_affiliation = pd.read_pickle('extra_info_affiliation_CS5099.pkl')
read_coredata = pd.read_pickle('extra_info_coredata_CS5099.pkl')
df_current_extra_info = pd.DataFrame()
df_current_extra_info['affiliation'] = read_affiliation
df_current_extra_info['coredata'] = read_coredata
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
74297,"[{'affiliation-city': 'Wageningen', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
74298,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '12 ..."
74299,"{'affiliation-city': 'Adelaide', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85091772561', ..."
74300,"[{'affiliation-city': 'San Diego', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '6',..."


In [3]:
df_current_extra_info.isnull().sum()

affiliation    14615
coredata       13897
dtype: int64

Thusly, all rows which contain "None" values are dropped and the DataFrame is reindexed. 

In [4]:
df_combined = df_current_extra_info.dropna()
df_combined = df_combined.reset_index(drop=True)
df_combined

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
3,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
4,"{'affiliation-city': 'Brighton', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85091848821', ..."
...,...,...
59682,"[{'affiliation-city': 'Wageningen', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
59683,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '12 ..."
59684,"{'affiliation-city': 'Adelaide', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85091772561', ..."
59685,"[{'affiliation-city': 'San Diego', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '6',..."


The following functions support the creation of DataFrames based on the columns affiliation and coredata. 

In [5]:
def get_one_entry(dic):
    df_affiliation_holder = pd.DataFrame(dic.items()).T
    df_affiliation_holder.columns = df_affiliation_holder.iloc[0]
    df_affiliation_holder = df_affiliation_holder.drop(df_affiliation_holder.index[0])
    return df_affiliation_holder

In [6]:
def get_various_entries(dic):
    df_affiliation_holder = pd.DataFrame.from_dict(dic, orient='columns')
    return df_affiliation_holder

The next cell creates the DataFrame which focus on affiliation data. 

In [7]:
%%time
df_affiliation = pd.DataFrame()
df_affiliation_holder = pd.DataFrame()

for i in df_combined['affiliation']:
    string_holder = str(i)
    if string_holder[0] == "[":
        df_affiliation_holder = get_various_entries(i)
    else: 
        df_affiliation_holder = get_one_entry(i)
    df_affiliation = pd.concat([df_affiliation_holder, df_affiliation],ignore_index=True)
df_affiliation

Wall time: 23min 4s


,affiliation-city,affilname,affiliation-country
0,Taipei,Ming Chi University of Technology,Taiwan
1,Taipei,National Taiwan University of Science and Tech...,Taiwan
2,Chung-Li,Yuan Ze University,Taiwan
3,San Diego,"University of California, San Diego",United States
4,Pittsburgh,University of Pittsburgh,United States
...,...,...,...
198889,New Delhi,Jamia Millia Islamia,India
198890,New Delhi,Indraprastha Apollo Hospitals,India
198891,None,Patan Academy of Health Sciences-School of Med...,Nepal
198892,Valhalla,New York Medical College,United States


In [8]:
# %%time
# df_coredata = pd.DataFrame()
# df_coredata_holder = pd.DataFrame()

# for i in df_combined['coredata']:
#     string_holder = str(i)
#     if string_holder[0] == "[":
#         df_coredata_holder = get_various_entries(i)
#     else: 
#         df_coredata_holder = get_one_entry(i)
#     df_coredata = pd.concat([df_coredata_holder, df_coredata],ignore_index=True)
# df_coredata

Leading figures where the researchers came from. One publication can have multiple affiliations. 

Leading countries

In [9]:
ser_country = df_affiliation['affiliation-country']
ser_country

0                Taiwan
1                Taiwan
2                Taiwan
3         United States
4         United States
              ...      
198889            India
198890            India
198891            Nepal
198892    United States
198893            India
Name: affiliation-country, Length: 198894, dtype: object

In [10]:
ser_country.value_counts()

United States               43536
China                       22952
United Kingdom              14702
Italy                       10504
France                       8548
                            ...  
North Korea                     1
Central African Republic        1
Dominica                        1
Guinea-Bissau                   1
Saint Lucia                     1
Name: affiliation-country, Length: 191, dtype: int64

In [22]:
# Due to ethical constraints the city and the name of the institution are removed before saving to the storage
del df_affiliation['affiliation-city']
del df_affiliation['affilname']

In [23]:
df_affiliation

,affiliation-country
0,Taiwan
1,Taiwan
2,Taiwan
3,United States
4,United States
...,...
198889,India
198890,India
198891,Nepal
198892,United States


In [24]:
%%time
df_coredata = pd.DataFrame()
df_coredata_holder = pd.DataFrame()

for i in df_combined['coredata']:
    string_holder = str(i)
    if string_holder[0] == "[":
        df_coredata_holder = get_various_entries(i)
    else: 
        df_coredata_holder = get_one_entry(i)
    df_coredata = pd.concat([df_coredata_holder, df_coredata],ignore_index=True)
df_coredata

Wall time: 16 ms


,srctype,prism:issueIdentifier,eid,prism:coverDate,prism:aggregationType,prism:url,subtypeDescription,dc:creator,link,prism:publicationName,...,prism:pageRange,dc:title,prism:endingPage,openaccess,openaccessFlag,prism:doi,prism:issn,prism:startingPage,dc:identifier,dc:publisher
0,j,230,2-s2.0-85092733043,2020-10-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"{'author': [{'ce:given-name': 'Carmina', 'pref...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Journal of the Nepal Medical Association,...,744-750,Mental wellbeing during the lockdown period fo...,750,1,true,10.31729/jnma.5498,00282715,744,SCOPUS_ID:85092733043,Nepal Medical Associationinfo@nma.org.np


In [25]:
# Due to ethical constraints the information linke to the creator (author) of the paper are removed before saving to the storage
del df_coredata['dc:creator']

In [27]:
df_coredata

,srctype,prism:issueIdentifier,eid,prism:coverDate,prism:aggregationType,prism:url,subtypeDescription,link,prism:publicationName,source-id,...,prism:pageRange,dc:title,prism:endingPage,openaccess,openaccessFlag,prism:doi,prism:issn,prism:startingPage,dc:identifier,dc:publisher
0,j,230,2-s2.0-85092733043,2020-10-01,Journal,https://api.elsevier.com/content/abstract/scop...,Article,"[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",Journal of the Nepal Medical Association,15831,...,744-750,Mental wellbeing during the lockdown period fo...,750,1,true,10.31729/jnma.5498,00282715,744,SCOPUS_ID:85092733043,Nepal Medical Associationinfo@nma.org.np
